In [2]:
import numpy as np
import libpysal as ps
from mgwr.gwr import GWR, MGWR
from mgwr.sel_bw import Sel_BW
import geopandas as gp
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd

In [28]:
def addcentroids_toshp(shp, geom_col, file_path):
    '''add centroid x y info to shp table'''
    centroids = shp.centroid
    for index, row in shp.iterrows():
        shp.loc[index, 'cen_x'] = centroids.iloc[index].x
        shp.loc[index, 'cen_y'] = centroids.iloc[index].y
    shp.to_file(file_path)
    return shp

In [ ]:
fl_counties = gp.read_file('c:\\Users\\Andy\\Downloads\\leukemia_fl\\leukemia_fl.shp') 
fl_counties_3857 = fl_counties.to_crs(epsg=3857)
# fl_data =  pd.read_csv(ps.examples.get_path('c:\\Users\\Andy\\Downloads\\leukemia_fl\\leukemia_fl.csv')) need to export a csv from postgis
#fig, ax = plt.subplots(figsize=(10,10))
#fl_counties.plot(ax=ax, **{'edgecolor':'black', 'facecolor':'white'})
#fl_counties.centroid.plot(ax=ax, c='black')
print(fl_counties)
print(fl_counties_3857)


In [60]:
fl_countiesxy = addcentroids_toshp(fl_counties_3857, 'geometry', 'c:\\Users\\Andy\\Downloads\\leukemia_fl\\leukemia_fl2.shp')
#print(fl_countiesxy)
#type(fl_countiesxy)

In [ ]:
#join population data to gdf
fl_pop = pd.read_csv('c:\\Users\\Andy\\Downloads\\fl_pop.csv', dtype={'COUNTY': str})
fl_pop = fl_pop.query('YEAR == 5')
fl_pop['YEAR'] = 2023
#print(fl_pop)
fl_pop.to_csv('c:\\Users\\Andy\\Downloads\\fl_pop2.csv', index=False)
fl_countiesxy= fl_countiesxy.merge(fl_pop[['COUNTY', 'POPESTIMATE']], left_on='FIPS', right_on='COUNTY', how='left')
fl_countiesxy.to_file('c:\\Users\\Andy\\Downloads\\leukemia_fl\\leukemia_fl3.shp')
fl_countiesxy.head() 

In [75]:
fl_countiesxy['area_sq_km'] = fl_countiesxy['geometry'].area / 1e6 
# Calculate population density (people per square kilometer)
fl_countiesxy['ppl_per_sqkm'] = fl_countiesxy['POPESTIMATE'] / fl_countiesxy['area_sq_km']
fl_countiesxy['ppl_per_sqkm'] = fl_countiesxy['ppl_per_sqkm'].round(0).astype(int)
fl_countiesxy= fl_countiesxy.drop('pop_density',axis=1)
fl_countiesxy.to_file('c:\\Users\\Andy\\Downloads\\leukemia_fl\\leukemia_fl4.shp')

fl_countiesxy.head()



C:\Users\Andy\AppData\Local\Temp\ipykernel_19116\1121346757.py:6: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  fl_countiesxy.to_file('c:\\Users\\Andy\\Downloads\\leukemia_fl\\leukemia_fl4.shp')


,NAME,FIPS,INCIDENCE_,AVG_ANNUAL,PERCENT_AD,PERCENT_OV,geometry,cen_x,cen_y,COUNTY,POPESTIMATE,area_sq_km,ppl_per_sqkm
0,Brevard,009,21.1,196,16.3,67.4,"POLYGON ((-9015459.431 3326480.833, -9015174.2...",-8.987088e+06,3.286203e+06,009,643979,4341.836676,148
1,Columbia,023,17.7,17,23.9,73.4,"POLYGON ((-9217131.720 3495022.348, -9217134.0...",-9.197380e+06,3.532487e+06,023,73063,2789.144396,26
2,Gadsden,039,14.1,9,15.1,72.7,"POLYGON ((-9454577.975 3581801.262, -9454355.3...",-9.419146e+06,3.578267e+06,039,43833,1853.111565,24
3,Highlands,055,14.9,30,16.5,65.7,"POLYGON ((-9079672.521 3170188.819, -9079639.5...",-9.054843e+06,3.166474e+06,055,107614,3645.974628,30
4,Jefferson,065,0.0,3 or fewer,17.1,65.0,"POLYGON ((-9359323.000 3519664.892, -9359276.6...",-9.339170e+06,3.559953e+06,065,15450,2142.135641,7
